### Alignment

#### naiive 

In [1]:
def readGenome(filename) :
    genome = ''
    with open (filename, 'r') as f:
        for line in f:
            #ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('phix.fa')

In [9]:
def naive(p, t): # p:pattern ; t: text
    occurrences = [] # initiate an empty list
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j] :
                match = False # a character doesnt match
                break # move on to next i
        if match:
            occurrences.append(i)
    return occurrences
    

In [10]:
p = 'AG'
t = 'AGAGAGAG'
naive(p, t)

[0, 2, 4, 6]

Generate artificial random sequences by taking subsequences of genome 

In [13]:
import random
def generateReads(genome, numReads, readLen):
    '''Generate reads from random positions in the given genome.'''
    reads = []
    for _ in range(numReads): # numReads is number of reads
        start = random.randint(0, len(genome) - readLen -1)
        reads.append(genome[start : start + readLen])
    return reads

In [14]:
reads = generateReads(genome, 100, 100) # 100 reads with length of 100

In [15]:
reads

['GGTTATATTGACCATGCCGCTTTTCTTGGCACGATTAACCCTGATACCAATAAAATCCCTAAGCATTTGTTTCAGGGTTATTTGAATATCTATAACAACT',
 'TCGTTATGGTTTCCGTTGCTGCCATCTCAAAAACATTTGGACTGCTCCGCTTCCTCCTGAGACTGAGCTTTCTCGCCAAATGACGACTTCTACCACATCT',
 'TTATCGCTGGCGACTCCTTCGAGATGGACGCCGTTGGCGCTCTCCGTCTTTCTCCATTGCGTCGTGGCCTTGCTATTGACTCTACTGTAGACATTTTTAC',
 'AAGGTAAAAAACGTTCTGGCGCTCGCCCTGGTCGTCCGCAGCCGTTGCGAGGTACTAAAGGCAAGCGTAAAGGCGCTCGTCTTTGGTATGTAGGTGGTCA',
 'CTTCTGTTACTCCAGCTTCTTCGGCACCTGTTTTACAGACACCTAAAGCTACATCGTCAACGTTATATTTTGATAGTTTGACGGTTAATGCTGGTAATGG',
 'GCGTAAAGGCGCTCGTCTTTGGTATGTAGGTGGTCAACAATTTTAATTGCAGGGGCTTCGGCCCCTTACTTGAGGATAAATTATGTCTAATATTCAAACT',
 'AAGTTAATGGATGAATTGGCACAATGCTACAATGTGCTCCCCCAACTTGATATTAATAACACTATAGACCACCGCCCCGAAGGGGACGAAAAATGGTTTT',
 'CTGTAGGCATGGGTGATGCTGGTATTAAATCTGCCATTCAAGGCTCTAATGTTCCTAACCCTGATGAGGCCGCCCCTAGTTTTGTTTCTGGTGCTATGGC',
 'CAATCGCCGCCAGTTAAATAGCTTGCAAAATACGTGGCCTTATGGTTACAGTATGCCCATCGCAGTTCGCTACACGCAGGACGCTTTTTCACGTTCTGGT',
 'ACGCAGAATCAGCGGTATGGCTCTTCTCATATTGGCGCTACTGCAAAGGATAT

In [18]:
numMatched = 0
for r in reads: # for each read in reads
    matches = naive(r, genome) # find the number of matches for each read
    if len(matches) > 0: # if there is matches
        numMatched += 1 # there is 1 more pattern that matched 
print('%d / %d reads matches exactly!' % (numMatched, len(reads)))

100 / 100 reads matches exactly!


### Real Read

In [23]:
def readFastq(fileName):
    sequences = []
    qualities = []
    with open(fileName) as fh:
        while True:
            fh.readline() # skip name line
            seq = fh.readline().rstrip() # read base sequence
            if len(seq) == 0:
                break
            fh.readline() # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [27]:
phix_reads, _ = readFastq('ERR266411_1.first1000.fastq.txt')

In [29]:
numMatched = 0
n = 0 # keeping track of number of sequences
for r in phix_reads: # for each sequences in phix_reads
    matches = naive(r[:30], genome)
    n += 1 
    if len(matches) > 0:
        numMatched += 1
print('%d/ %d matched!' % (numMatched, n))

459/ 1000 matched!


**low matched number because of sequencing errors, and the genome is double stranded**

### Matching both forward and reverse strand 

In [30]:
def reverseComplement(s):
    complement = {'A':'T', 'C':'G', 'G':'C', 'T':'A', 'N':'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [31]:
numMatched = 0
n = 0 # keeping track of number of sequences
for r in phix_reads: # for each sequences in phix_reads
    r = r[:30]
    matches = naive(r[:30], genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1 
    if len(matches) > 0:
        numMatched += 1
print('%d/ %d matched!' % (numMatched, n))

932/ 1000 matched!
